In [19]:
import numpy as np
import cv2
from collections import deque
from datetime import datetime as time
import tkinter as tk

In [20]:
# Choose training or testing dataset
root = tk.Tk()
v = tk.StringVar()

tk.Label(root, text='Choose dataset : ', justify = tk.CENTER, padx = 20).pack()
tk.Radiobutton(root, text="train", indicatoron = 0, padx = 20, variable=v, value='train').pack(anchor=tk.W)
tk.Radiobutton(root, text="test", indicatoron = 0, padx = 20, variable=v, value='test').pack(anchor=tk.W)
root.mainloop()

choice = v.get()

In [21]:
if choice == 'train':
    # Choose authentic or forged dataset
    root = tk.Tk()
    v = tk.StringVar()

    tk.Label(root, text='Choose dataset : ', justify = tk.CENTER, padx = 20).pack()
    tk.Radiobutton(root, text="authentic", indicatoron = 0, padx = 20, variable=v, value='authentic').pack(anchor=tk.W)
    tk.Radiobutton(root, text="forged", indicatoron = 0, padx = 20, variable=v, value='forged').pack(anchor=tk.W)
    root.mainloop()

    trainChoice = v.get()

In [22]:
# Define the upper and lower boundaries for a color to be considered "Blue"
blueLower = np.array([100, 60, 60])
blueUpper = np.array([140, 255, 255])

# Define a 5x5 kernel for erosion and dilation
kernel = np.ones((5, 5), np.uint8)

# Setup deques to store separate colors in separate arrays
bpoints = [deque(maxlen=512)]

bindex = 0

color = (255, 0, 0)
colorIndex = 0

In [23]:
# Setup the Paint interface
paintWindow = np.zeros((471,636,3)) + 255

cv2.namedWindow('Paint', cv2.WINDOW_AUTOSIZE)

In [24]:
# Load the video
camera = cv2.VideoCapture(0)

# Keep looping
while True:
    # Grab the current paintWindow
    (grabbed, frame) = camera.read()
    frame = cv2.flip(frame, 1)
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Add the coloring options to the frame
    frame = cv2.rectangle(frame, (40,1), (140,65), (122,122,122), -1)
    frame = cv2.rectangle(frame, (160,1), (255,65), color, -1)
    cv2.putText(frame, "CLEAR ALL", (49, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
    cv2.putText(frame, "BLUE", (185, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
   
    # Check to see if we have reached the end of the video
    if not grabbed:
        break

    # Determine which pixels fall within the blue boundaries and then blur the binary image
    blueMask = cv2.inRange(hsv, blueLower, blueUpper)
    blueMask = cv2.erode(blueMask, kernel, iterations=2)
    blueMask = cv2.morphologyEx(blueMask, cv2.MORPH_OPEN, kernel)
    blueMask = cv2.dilate(blueMask, kernel, iterations=1)

    # Find contours in the image
    (_, cnts, _) = cv2.findContours(blueMask.copy(), cv2.RETR_EXTERNAL,
    	cv2.CHAIN_APPROX_SIMPLE)
    center = None

    # Check to see if any contours were found
    if len(cnts) > 0:
    	# Sort the contours and find the largest one -- we
    	# will assume this contour correspondes to the area of the bottle cap
        cnt = sorted(cnts, key = cv2.contourArea, reverse = True)[0]
        # Get the radius of the enclosing circle around the found contour
        ((x, y), radius) = cv2.minEnclosingCircle(cnt)
        # Draw the circle around the contour
        cv2.circle(frame, (int(x), int(y)), int(radius), (0, 255, 255), 2)
        # Get the moments to calculate the center of the contour (in this case Circle)
        M = cv2.moments(cnt)
        center = (int(M['m10'] / M['m00']), int(M['m01'] / M['m00']))

        if center[1] <= 65:
            if 40 <= center[0] <= 140: # Clear All
                bpoints = [deque(maxlen=512)]
                bindex = 0
                paintWindow[67:,:,:] = 255
                
            else:
                    colorIndex = 0 # Blue
        else :
            bpoints[bindex].appendleft(center)
   
    # Append the next deque when no contours are detected (i.e., bottle cap reversed)
    else:
        bpoints.append(deque(maxlen=512))
        bindex += 1

    # Draw lines of Blue color
    for j in range(len(bpoints)):
        for k in range(1, len(bpoints[j])):
            if bpoints[j][k - 1] is None or bpoints[j][k] is None:
                continue
            cv2.line(frame, bpoints[j][k - 1], bpoints[j][k], color, 2)
            cv2.line(paintWindow, bpoints[j][k - 1], bpoints[j][k], color, 2)

    # Show the frame and the paintWindow image
    cv2.imshow("Tracking", frame)
    cv2.imshow("Paint", paintWindow)

	# If the 'q' key is pressed, stop the loop
    if cv2.waitKey(1) & 0xFF == ord("q"):
        
        if choice == 'train':
            if trainChoice == 'forged':
                label = 'dataset/train/forged/img '
            else:
                label = 'dataset/train/authentic/img '
        else:
            label = 'dataset/test/img '
            
        timeStamp = str(time.now())[:-7].replace(':','-')
        label = label + timeStamp + '.png'
        cv2.imwrite(label, paintWindow)
        img = cv2.imread(label)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        final = cv2.subtract(255, gray)
        cv2.imwrite(label, final)
        break

# Cleanup the camera and close any open windows
camera.release()
cv2.destroyAllWindows()